# Make your data ready for QGIS

We start with some imports, than you set your project folder and the python script will generate the GeoTiff file you need for QGIS. All steps will print some info for you :)
Execute all steps after each other. Your input is needed on **Step 3**!

## Step 1: Imports

Make your script ready for the processing

In [ ]:
import os
import shutil
import logging
from pathlib import Path
from typing import List, Optional
from osgeo import gdal
import time

# Configure logging for Jupyter
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')
logger = logging.getLogger(__name__)

# Configure GDAL for better performance
gdal.SetConfigOption('GDAL_CACHEMAX', '512')  # Increase cache to 512MB
gdal.SetConfigOption('GDAL_NUM_THREADS', 'ALL_CPUS')  # Use all CPU cores
gdal.SetConfigOption('GDAL_DISABLE_READDIR_ON_OPEN', 'EMPTY_DIR')  # Faster file opening
gdal.UseExceptions()
target_epsg = "EPSG:25833"

print("✅ Setup complete!")

## Step 2: This is the actual code, execute once so the script can than be started

In [ ]:
def quick_convert(input_folder: str, target_epsg: str = "EPSG:25833") -> Optional[Path]:
    
    input_path = Path(input_folder)
    temp_folder = input_path / "temp" # Creates a temporary folder, which can be deleted afterwards
    temp_folder.mkdir(exist_ok=True)
    
    try:
        # Find XYZ files
        xyz_files = list(input_path.rglob("*.xyz"))
        print(f"📊 Found {len(xyz_files)} XYZ files")
        
        if not xyz_files:
            print("❌ No XYZ files found!")
            return None
        
        # Convert and transform in one step
        transformed_files = []
        for i, xyz_file in enumerate(xyz_files, 1):
            print(f"⚙️  Processing {i}/{len(xyz_files)}: {xyz_file.name}")
            
            # Direct conversion with projection
            output_path = temp_folder / f"processed_{xyz_file.stem}.tif"
            
            # Convert XYZ to GeoTIFF with target projection
            gdal.Translate(
                str(output_path),
                str(xyz_file),
                outputSRS=target_epsg,
                format="GTiff",
                creationOptions=["COMPRESS=LZW", "TILED=YES"]
            )
            
            # Transform projection if needed
            final_path = temp_folder / f"final_{xyz_file.stem}.tif"
            gdal.Warp(
                str(final_path),
                str(output_path),
                dstSRS=target_epsg,
                format="GTiff",
                creationOptions=["COMPRESS=LZW", "TILED=YES"],
                multithread=True
            )
            
            transformed_files.append(final_path)
            os.remove(output_path)  # Clean up intermediate file
        
        # Merge all files
        merged_output = input_path / "merged_dem.tif"
        print(f"🔄 Merging {len(transformed_files)} files...")
        
        gdal.Warp(
            str(merged_output),
            [str(f) for f in transformed_files],
            format="GTiff",
            creationOptions=["COMPRESS=LZW", "TILED=YES", "BIGTIFF=IF_SAFER"],
            multithread=True
        )
        
        # Cleanup
        shutil.rmtree(temp_folder)
        
        print(f"✅ Success! Output: {merged_output}")
        print(f"📏 File size: {merged_output.stat().st_size / (1024*1024):.2f} MB")
        
        return merged_output
        
    except Exception as e:
        print(f"❌ Error: {e}")
        # Cleanup on error
        if temp_folder.exists():
            shutil.rmtree(temp_folder)
        return None

print("🚀 Quick convert function ready!")

## Step 3: Set your folder and start conversion

Put your folder location inside of the brackets

In [ ]:
# Set your project folder -  the rest will be done automatically

input_folder = "/home/example_folder/"


# Validate path to see if your folder exists
input_folder = Path(input_folder)
if not input_folder.exists():
    raise FileNotFoundError(f"Input folder does not exist: {input_folder}")

print(f"📁 Input folder: {input_folder}")



## Step 4: Time to start the actual transformation from XYZ files to GeoTIFF

Depending on the area you chose this may take a while. 
16km² take around 30s on a Ryzen 3600 with 16GB of RAM. 

There is a little timer and frequent prints of which file is currently processed

In [ ]:
start_time = time.time() # Starts your timer

result = quick_convert(input_folder, target_epsg) # Converts your files and merges them to one TIFF

end_time = time.time() 
processing_time = end_time - start_time

print(f"\n⏱️  Processing time: {processing_time:.2f} seconds")

if result:
    print(f"🎯 Ready for QGIS: {result}")
else:
    print("💥 Conversion failed - check the logs above")